## Convection / flow modelling (with underworld)


  - Intro to underworld - quick overview of the basics
  - Solvers: Global flow from local source of buoyancy or bc. (Why MG)
  - Analytic solutions in Underworld and how to use them to estimate / test 
  - Scaling & Rayleigh number
  - Rayleigh number and Nusselt number
  - Meshes / interpolation ... checkpointing
  - Tracers / strain markers, analysis
  - Thermal / chemical convection and more 


### The basics

Underworld is configured to solve a couple Stokes / advection-diffusion problem on a finite element mesh. 

The basic tools we need are:
    
   - The mesh itself
   - Some unknowns living on the mesh
   - Some boundary conditions
   - A driving term for the flow
   - Constitutive terms (viscosity, elasticity, plasticity)
   - A way to solve the equations
   - Some means of analysing the result
   - A means of timestepping
     
### Equations

The dimensionless (indicated by primes) Navier-Stokes equation can be written

$$
	\frac{1}{\textrm{Pr}} \frac{D \mathbf{v}'  }{Dt' }  =
	\acute{\nabla}^2  \mathbf{v}'  -  \acute{\nabla} p' + \textrm{ Ra} T' \hat{\mathbf{z}}
$$

We will work in the infinite Prandtl number limit in which intertial terms vanish, and we will also assume incompressibility

$$
\nabla^2  \mathbf{v}  -  {\nabla} p = -\textrm{ Ra} T \hat{\mathbf{z}}
$$
$$
\nabla \cdot \mathbf{v} = 0
$$

The buoyancy force in the Stokes equation includes (but is not limited to) thermal variations which are themselves coupled to the solution of the Stokes equation through the thermal advection term.

$$
			\frac{\partial T}{\partial t} + \mathbf{v}\cdot \nabla T -
					\kappa \nabla^2 T = \frac{H}{C_p}
$$

   
### Hands on  - quasi-static problems
   
The simplest problem we can set up is a boundary driven flow


  - [Lid Driven Flow Notebook](020-LidDrivenFlow.ipynb) 
     - Exercise: [Point Driving Velocity](021-Exercise-PointSource.ipynb) 


The next thing to try is a buoyancy driven flow 


  - [Thermally-driven flow](030-ThermalBuoyancy.ipynb)
  

Which works pretty much as expected - by adding an appropriate function to the buoyancy term in the Stokes equation template.


### Hands on - time-stepping

Let's look at how to make a time-dependent problem. We start by looking at 
some passive tracers swirling around in a lid-driven flow.

   - [Lid Driven Flow with Tracers](040-LidDrivenFlowPlusTracers.ipynb) 
      - Exercise (take away): Kellogg & Turcotte, 1990 
      - Exercise: Streak plots    

Nice, but now we should let the flow be buoyancy driven and allow the buoyancy to drive the flow.

   - [Rayleigh Bénard Convection](050-RayleighBenardConvection.ipynb)
   - [Rayleigh Bénard Convection - pt2 ](051-RayleighBenardConvection-pt2.ipynb)
       - Exercise (take away): Nu - Ra and Vrms - Ra plots (again)
   - [Rayleigh Bénard Convection - pt3](052-RayleighBenardConvection-pt3.ipynb)
       - Exercise (take away): Arrhenius rheology and non-linear viscosity


## What have we not covered ?

There are many aspects of the `underworld` code and of modelling in general that we have not dealt with here, or have only touched upon in a general way. Some of these issues apply to all the different numerical methods and tools you might come across.

   - **Where can I find out more ?**  _Python codes include 'docstrings' which can be quite helpful and you should always take a look. uw.help() grabs the docstrings for you and leaves out some of the developer-specific complexities._

   - **Treat the code like a black box, how do you know when to trust the output ?** _In `underworld`, we treat this question very seriously and we bundle a whole collection of analytic Stokes flow problems in with the code ( `uw.help(uw.fn.analytic)` )_
   
   - **What are the trade-offs between resolution, accuracy, choice of solvers, speed, parallel performance etc etc ?** _You probably want to find the answer(s) to that question out for yourself, I just want to make you aware that it is one of the key questions in modeling. Having analytic or approximate solutions to hand is very helpful in this case. Also seeing if the results **converge** to a specific answer as you put more resources into the solution._ 

   - **Particles and material history.** _One of the reasons `underworld` was written was to look at problems of complex fluids - ones where the material history is relevant to the constitutive behaviour (e.g. anisotropy, viscoelasticity, strain-softening ...). There are examples of these in the `underworld` user guide and tutorials._
   
   - **Parallel ?** _Most of the time, the python scripts will run fine in parallel, some of the more convoluted things we saw in the examples are the result of guiding you to parallel safe methods (e.g. the `uw.fn.view.maxmin` which works in parallel when numpy.max() does not). Also, the whole `gLucifer` visualisation framework is designed for use in parallel. Have a look at the examples, they all work in parallel._
   
   - **Saving data.** _You can save and read back all field and swarm variables and this all works in parallel. The file formats are hdf5 and these can be read later (in serial if you can't process your results easily in parallel). This is important as you may / will want to checkpoint and restart jobs that take a long time to run. **Warning:** you will have to make sure your scripts handle whatever branches arise from restarting as we cannot snapshot your scripts and save them._
   
   - **Workflow.** _There are many, many python packages available which will just work with underworld. For example, to allow you to set up complicated pattens of particles using projections, mapping data to material properties or comparing observations and model outputs. Many are already built-in to `scipy`. You might end up with some issues in parallel if you are not careful (e.g. `scipy`'s fft routines), but perhaps you can create a workflow in which this processing is handled in serial, saves hdf5 information, and is then used to initialise a parallel script._



In [14]:
import underworld as uw
from underworld import function as fn
import glucifer
import numpy as np

In [15]:
print uw.help(uw)

Object docstring:


Underworld2 is a python-friendly version of the Underworld geodynamics
code which provides a programmable and flexible front end to all the
functionality of the code running in a parallel HPC environment. This
gives signficant advantages to the user, with access to the power of
python libraries for setup of complex problems, analysis at runtime,
problem steering, and coupling of multiple problems.

Underworld2 is integrated with the literate programming environment of
the jupyter notebook system for tutorials and as a teaching tool for
solid Earth geoscience.

Underworld is an open-source, particle-in-cell finite element code
tuned for large-scale geodynamics simulations. The numerical algorithms
allow the tracking of history information through the high-strain
deformation associated with fluid flow (for example, transport of the
stress tensor in a viscoelastic, convecting medium, or the advection of
fine-scale damage parameters by the large-scale flow). The finite


In [16]:
## Tab completion:  fn.
## gives you a helpful list with all the available functions
## but it leaves out the _hidden_ ones which the following does not

print dir(fn.math)

['_Function', '__builtins__', '__doc__', '__file__', '__name__', '__package__', '_cfn', 'abs', 'acos', 'acosh', 'asin', 'asinh', 'atan', 'atanh', 'cos', 'cosh', 'dot', 'erf', 'erfc', 'exp', 'input', 'log', 'log10', 'log2', 'pow', 'sin', 'sinh', 'sqrt', 'tan', 'tanh', 'uw']


In [17]:
print uw.help(fn.analytic.SolCx, toScreen=False)


    SolCx is a 2-dimensional analytical solution to the Stokes' flow  equations.
    The boundary conditions are free-slip everywhere on a unit domain.
    There is a viscosity jump in the x direction at $x=x_c$.
    The flow is driven by a temperature field represented by the density, $ \rho $, as follows:
    $$
         \rho = -\sigma \sin (n_z \pi z) \cos (n_x \pi x).
    $$

    Parameters:

    The variable parameters of this solution are:

      - density parameter: $ \sigma $
      - viscosities: $\eta_A$ and $\eta_B$
      - viscosity jump location: $x_c$.
      - wave number in z domain: $ n_z $. ($n_z$ may be non-integer) ((NOT AVAILABLE IN PYTHON))
      - wave number in x domain: $ n_x $. ($n_x$ must be integer)

    


In [18]:
print uw.help(uw.systems.Stokes)

Object docstring:


    This class provides functionality for a discrete representation
    of the incompressible Stokes equation.

    Specifically, the class uses a mixed finite element method to
    construct a system of linear equations which may then be solved
    using an object of the underworld.system.Solver class.

    The underlying element types are determined by the supporting
    mesh used for the 'velocityField' and 'pressureField' parameters.

    Parameters
    ----------
    velocityField : underworld.mesh.MeshVariable
        Variable used to record system velocity.
    pressureField : underworld.mesh.MeshVariable
        Variable used to record system pressure.
    fn_viscosity : underworld.function.Function
        Function which reports a viscosity value.
        Function must return scalar float values.
    fn_bodyforce : underworld.function.Function, default=None.
        Function which reports a body force for the system.
        Function must return float value